<a href="https://colab.research.google.com/github/mjbarents-bfa/machine-learning-example/blob/main/customer_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Machine Learning voorbeeld**
In dit voorbeeld gaan we gebruik maken van een '**Ramdom Forrest Classifier**' en data vanuit een **klantenbestand**.


> Een random forest classifier is een machine learning-algoritme dat wordt gebruikt voor classificatietaken. Het maakt meerdere beslisbomen met willekeurige data en eigenschappen. Dan combineert het de uitkomsten van deze bomen voor een betere voorspelling.

Het algoritme gaat patronen proberen te leren uit het klantenbestand, om zo voorspellingen te kunnen maken over nieuwe klanten.






### **Stap 1**
Laten we aan de slag gaan, eerst moeten we wat modules importeren die we later nodig zullen hebben. Druk hiervoor op het 'uitvoeren' knopje linksboven in het vakje hieronder.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

### **Stap 2**

Top! Dan gaan we nu de data inladen, in dit geval een klantenbestand met bepaalde eigenschappen van klanten. Onder deze eigenschappen bestaat ook een '**Spending Score**'. Deze score wordt toegekend aan klanten op basis van hun koopgedrag.

In [ ]:
csv_url = 'https://raw.githubusercontent.com/mjbarents-bfa/machine-learning-example/main/Customers.csv'
df = pd.read_csv(csv_url)
df

### **Stap 3**
We hebben geluk, de data was van tevoren al goed gestructureerd waardoor we de data zonder problemen meteen al in konden laden, en meteen al een mooie tabel zien. Betekent dit dan dat al onze data ook correct is?

Dat valt nog te bezien...

Laten we bijvoorbeeld even kijken of er klanten tussen zitten die al voor hun 10e aan het werk waren:

In [ ]:
filtered_df = df[(df['Age'] - df['Work_Experience']) < 10]
filtered_df

###**Stap 4**
Dokter of artiest worden voor je 10e levensjaar? Dat is knap, misschien wel iets te knap. Laten we voor de zekerheid klanten die zogenaamd voor hun 18e al werk hadden, eruit filteren.

In [ ]:
df = df[(df['Age'] - df['Work_Experience']) < 18]

### **Stap 5**
Wellicht zijn er nog meer 'incorrecte' klantengegevens, maar voor nu laten we het hierbij. Laten we beginnen met het trainen van ons algoritme, in het vakje hieronder trainen we de **Random Forrest Classifier** op onze klantgegevens.

In [ ]:
# Load the DataFrame
train_df = df.copy()

# Splitting the data into features (X) and target (y)
X = train_df.drop(columns=['Spending_Score', 'ID'])  # Drop 'ID' as it's not useful for prediction
y = train_df['Spending_Score']

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=26)

# Train the model
rf_classifier.fit(X, y)

### **Stap 6**
Oeps, daar ging iets fout. Het algoritme werkt enkel met getallen, maar in het klantenbestand staat ook tekst zoals *'Male', 'Female', 'Other'*. Voordat we ons **Random Forrest Classifier** model kunnen trainen, zullen we tekst dus eerst om moeten zetten naar getallen. Laten we dat doen:

In [ ]:
def label_encoder(df, is_train = True):
    gender_map = {"Female": 0, "Male": 1, "Other": 2}
    married_map = {"No": 0, "Yes": 1}
    graduated_map = {"No": 0, "Yes": 1}
    profession_map = {
        "Artist": 0,
        "Healthcare": 1,
        "Engineer": 2,
        "Doctor": 3,
        "Lawyer": 4,
        "Entertainment": 5,
        "Executive": 6,
        "Homemaker": 7,
        "Marketing": 8
    }
    spending_score_map = {"Low": 0, "Average": 1, "High": 2}

    # Apply label encoding to the dataframe
    df['Gender'] = df['Gender'].map(gender_map)
    df['Married'] = df['Married'].map(married_map)
    df['Graduated'] = df['Graduated'].map(graduated_map)
    df['Profession'] = df['Profession'].map(profession_map)
    if is_train == True:
      df['Spending_Score'] = df['Spending_Score'].map(spending_score_map)
    return df

df = label_encoder(df)

### **Stap 7**
Top! Nu dat gedaan is gaan we opnieuw proberen het model te trainen:

In [ ]:
# Load the DataFrame
train_df = df.copy()

# Splitting the data into features (X) and target (y)
X = train_df.drop(columns=['Spending_Score', 'ID'])  # Drop 'ID' as it's not useful for prediction
y = train_df['Spending_Score']

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=26)

# Train the model
rf_classifier.fit(X, y)

### **Stap 8**
Onee, er gaat nogsteeds iets fout. In het klantenbestand zitten klanten die bepaalde waarden missen. Zo gaat dat nou eenmaal met 'echte ruwe data', daar zitten soms fouten in. Deze missende waarden worden gelezen als 'NaN' (*Not a Number*). Ons algoritme weet niet hoe het hiermee om moet gaan, dus geeft het een foutmelding.

Er zijn meerdere 'slimme' manieren om hier mee om te gaan, maar laten we deze gevallen er voor nu gewoon uit filteren:

In [ ]:
def dropNaN(df):
    for index, row in df.iterrows():
        if row.isnull().any():
            df.drop(index, inplace=True)
    return df

df = dropNaN(df)

### **Stap 9**
Drie maal is scheepsrecht, laten we het nog een keer proberen:

In [ ]:
def predict(new_instance):
  new_instance = label_encoder(new_instance, False)
  prediction = rf_classifier.predict(new_instance)
  predicted_spending_scores = [{0: "Low", 1: "Average", 2: "High"}[pred] for pred in prediction]
  return predicted_spending_scores

# Load the DataFrame
train_df = df.copy()

# Splitting the data into features (X) and target (y)
X = train_df.drop(columns=['Spending_Score', 'ID'])  # Drop 'ID' as it's not useful for prediction
y = train_df['Spending_Score']

# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=26)

# Train the model
rf_classifier.fit(X, y)

### **Stap 10**
Gefeliciteerd, het is gelukt! We hebben nu succesvol een **Random Forest Classifier** getrained op ons klantenbestand.

Nu kunnen we het model gebruiken om voor nieuwe klanten te voorspellen wat hun **spending score** zal gaan zijn. Op basis hiervan kunnen we in de toekomst specifieke klanten beter targetten.

Benieuwd naar de spending score die het model voorspelt voor jou, je collega of een andere denkbeeldige klant? Pas de code hieronder aan om het resultaat van ons model te zien!

In [ ]:
nieuwe_klant = pd.DataFrame({
    "Gender": ["Male"], # Male/Female/Other
    "Married": ["Yes"], # Yes/No
    "Age": [60], # Any number
    "Graduated": ["Yes"], # Yes/No
    "Profession": ["Engineer"], # Artist/Healthcare/Engineer/Doctor/Lawyer/Executive/Homemaker/Marketing
    "Work_Experience": [10], # Any number
    "Family_Size": [2] # Any number
})

print("Het model voorspelt de volgende spending score:", predict(nieuwe_klant))